In [1]:
import sys

import cv2
from keras.models import load_model
import numpy as np
import pandas as pd

from utils.datasets import get_labels
from utils.inference import detect_faces
from utils.inference import load_detection_model
from utils.inference import make_face_coordinates
from utils.inference import draw_text
from utils.inference import draw_bounding_box
from utils.inference import apply_offsets

from utils.inference import load_image
from utils.preprocessor import preprocess_input


# parameters for loading data and images
#image_path = sys.argv[1]
emotion_model_path = '../trained_models/emotion_models/fer2013_mini_XCEPTION.102-0.66.hdf5'
gender_model_path = '../trained_models/gender_models/simple_CNN.81-0.96.hdf5'
emotion_labels = get_labels('fer2013')
gender_labels = get_labels('imdb')
font = cv2.FONT_HERSHEY_SIMPLEX

Using TensorFlow backend.


In [2]:
# hyper-parameters for bounding boxes shape
gender_offsets = (30, 60)
gender_offsets = (10, 10)
emotion_offsets = (20, 40)
emotion_offsets = (0, 0)

In [3]:
# loading models
face_detection = load_detection_model()
emotion_classifier = load_model(emotion_model_path, compile=False)
gender_classifier = load_model(gender_model_path, compile=False)

# getting input model shapes for inference
emotion_target_size = emotion_classifier.input_shape[1:3]
gender_target_size = gender_classifier.input_shape[1:3]

In [4]:
def analyze_image(image_path, frame_number):
    
    # loading images
    rgb_image = load_image(image_path, grayscale=False)
    gray_image = load_image(image_path, grayscale=True)
    gray_image = np.squeeze(gray_image)
    gray_image = gray_image.astype('uint8')
    detected_faces, score, idx = detect_faces(face_detection, gray_image)

    text = []
    
    for detected_face in detected_faces:

        face_coordinates = make_face_coordinates(detected_face)

        x1, x2, y1, y2 = apply_offsets(face_coordinates, gender_offsets)
        rgb_face = rgb_image[y1:y2, x1:x2]

        x1, x2, y1, y2 = apply_offsets(face_coordinates, emotion_offsets)
        gray_face = gray_image[y1:y2, x1:x2]

        try:
            rgb_face = cv2.resize(rgb_face, (gender_target_size))
            gray_face = cv2.resize(gray_face, (emotion_target_size))
        except:
            continue

        rgb_face = preprocess_input(rgb_face, False)
        rgb_face = np.expand_dims(rgb_face, 0)
        gender_prediction = gender_classifier.predict(rgb_face)
        gender_label_arg = np.argmax(gender_prediction)
        gender_text = gender_labels[gender_label_arg]

        gray_face = preprocess_input(gray_face, True)
        gray_face = np.expand_dims(gray_face, 0)
        gray_face = np.expand_dims(gray_face, -1)
        emotion_label_arg = np.argmax(emotion_classifier.predict(gray_face))
        emotion_text = emotion_labels[emotion_label_arg]

        if gender_text == gender_labels[0]:
            color = (0, 0, 255)
        else:
            color = (255, 0, 0)

        draw_bounding_box(face_coordinates, rgb_image, color)
        draw_text(face_coordinates, rgb_image, gender_text, color, 0, -20, 1, 2)
        draw_text(face_coordinates, rgb_image, emotion_text, color, 0, -50, 1, 2)
    
        print('gender_text', gender_text)
        print('emotion_text', emotion_text)
        
        line = ''
        line = str(frame_number) + ',' + str(gender_text) + ',' + str(emotion_text) 
        line = line + '\n'
        text += line 
        print('line', line)
        
        
        
        #with open(file_path,'wb') as file:
        #for line in text:
        #    file.write(line)
        #    file.write('\n')
    
    return text
    #bgr_image = cv2.cvtColor(rgb_image, cv2.COLOR_RGB2BGR)
    #cv2.imwrite('../images/predicted_test_image.png', bgr_image)

In [8]:
im_path2 = '/home/mayank/Desktop/gender-bias-detection/2017-RISS-Closing Lunch-Aug 9-135.JPG'
im_path = '/home/mayank/Desktop/gender-bias-detection/IMG_0139.JPG'
file_path = '/home/mayank/Desktop/gender-bias-detection/test.csv'
frame_number = 0
text = analyze_image(im_path, frame_number)

with open(file_path,'a') as file:
    for line in text:
        file.write(line)
        
text = analyze_image(im_path2, frame_number)

with open(file_path,'a') as file:
    for line in text:
        file.write(line)


('gender_text', 'man')
('emotion_text', 'happy')
('line', '0,man,happy\n')
('gender_text', 'man')
('emotion_text', 'happy')
('line', '0,man,happy\n')
('gender_text', 'woman')
('emotion_text', 'sad')
('line', '0,woman,sad\n')
('gender_text', 'man')
('emotion_text', 'happy')
('line', '0,man,happy\n')


In [9]:
data_base_path = '/home/mayank/Desktop/gender-bias-detection/trailer-data/'
frame_base_path = '/home/mayank/Desktop/gender-bias-detection/trailer-frames/'
pattern = "*.jpg"

df = pd.read_csv('/home/mayank/Desktop/gender-bias-detection/trailers_info.csv')

import glob, os

for index, row in df.iterrows():
    #print('row6', row[6])
    if (row[6] == 1):
        movie_name = row[3]
        #print('movie_name', movie_name)
        year = row[4]
        #print('year', year)
        new_data_folder_path = str(data_base_path) + str(year) + '-' + str(movie_name) + '/'
        #print('new_data_folder_path', new_data_folder_path)
        frame_path = frame_base_path + str(year) + '-' + str(movie_name) + '/'
        #print('frame_path', frame_path)
        
        if not os.path.exists(new_data_folder_path):
            os.makedirs(new_data_folder_path)
            
            csv_path = new_data_folder_path + str(year) + '-' + str(movie_name) + '.csv'
            #print('csv path', csv_path)
            
            for img_file in glob.glob(frame_path + "*.jpg"):
                print('img_file', img_file)
                head, tail = os.path.split(img_file)
                frame_number = str(int(filter(str.isdigit, tail)))
                print('frame_number', frame_number)
                
                
                text = analyze_image(img_file, frame_number)
                print('text', text)
                
                if os.path.exists(csv_path):
                    file_opening_mode = 'a' # append if already exists
                else:
                    file_opening_mode = 'w' # make a new file if not

                with open(csv_path,file_opening_mode) as csv_file:
                    for line in text:
                        csv_file.write(line)
                
                #break

In [45]:
# Concatening all csv files into one

import glob
import os

data_base_path = '/home/mayank/Desktop/gender-bias-detection/trailer-data/'
os.chdir(data_base_path)
result = glob.glob( '*/**.csv' )
#print('result', result)
print('len of result before', len(result))

# Removing empty files
result = [x for x in result if os.stat(x).st_size != 0]

print('len of result after', len(result))
               
globbed_files = []

for i in result:
    globbed_files.append(i.split('/')[1])

#print('globbed files', globbed_files)

data = [] # pd.concat takes a list of dataframes as an agrument

for csv in globbed_files:
    print('csv', csv)
    f = os.path.basename(csv)
    print('filename', filename)
    filename = f.split('.')[0]
    year = filename.split('-')[0]
    movie_name = filename.split('-')[1]
    csv_path = data_base_path + year + '-' + movie_name + '/' + csv
    print('csv_path', csv_path)
    frame = pd.read_csv(csv_path, header = None, sep = ',')
    frame.columns = ['frame_number', 'man/woman', 'emotion']
    frame['year'] = year
    frame['movie_name'] = movie_name
    frame.to_csv(csv_path, index=False)
    data.append(frame)
    #break

big_df = pd.concat(data, ignore_index=True) #dont want pandas to try an align row indexes
big_df.to_csv('/home/mayank/Desktop/gender-bias-detection/data.csv', sep = ',' , encoding='utf-8', index=False)

('len of result before', 860)
('len of result after', 853)
('csv', '2014-dedh_ishqiya.csv')
('filename', '2014-dedh_ishqiya')
('csv_path', '/home/mayank/Desktop/gender-bias-detection/trailer-data/2014-dedh_ishqiya/2014-dedh_ishqiya.csv')
('csv', '2010-love_sex_aur_dhokha.csv')
('filename', '2014-dedh_ishqiya')
('csv_path', '/home/mayank/Desktop/gender-bias-detection/trailer-data/2010-love_sex_aur_dhokha/2010-love_sex_aur_dhokha.csv')
('csv', '2017-newton.csv')
('filename', '2010-love_sex_aur_dhokha')
('csv_path', '/home/mayank/Desktop/gender-bias-detection/trailer-data/2017-newton/2017-newton.csv')
('csv', '2010-band_baaja_baaraat.csv')
('filename', '2017-newton')
('csv_path', '/home/mayank/Desktop/gender-bias-detection/trailer-data/2010-band_baaja_baaraat/2010-band_baaja_baaraat.csv')
('csv', '2016-the_blueberry_hunt.csv')
('filename', '2010-band_baaja_baaraat')
('csv_path', '/home/mayank/Desktop/gender-bias-detection/trailer-data/2016-the_blueberry_hunt/2016-the_blueberry_hunt.csv')
